In [1]:
# Architecture and loss functions inspired and adapted from https://github.com/ternaus/kaggle_dstl_submission
# And: https://github.com/amirdel/stanfordHacks
import os
import numpy as np

from keras import backend
from keras.models import Model

from keras.layers.merge import concatenate
from keras.layers import merge, Input, Conv2D, MaxPooling2D, UpSampling2D, Cropping2D, Flatten, Dense, BatchNormalization, Dropout

from keras.optimizers import Adam
from keras.backend import binary_crossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

from PIL import Image

from generator import batch_generator
from loss_functions import jaccard_coef_loss, jaccard_coef_int

Using TensorFlow backend.


# Model

In [2]:
# Loss function: sum of binary cross_entropy and jaccard index
# Architecture inspired from the U-Net paper (https://arxiv.org/pdf/1505.04597.pdf) 
def get_unet(num_channels, img_rows, img_cols):
    
    inputs = Input((img_rows, img_cols, num_channels))
    conv1 = Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv1)
    conv1 = BatchNormalization()(conv1)tc
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, 3, padding='same', kernel_initializer ='he_uniform', activation = 'elu')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, 3, padding='same', kernel_initializer ='he_uniform', activation = 'elu')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(512, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv5)
    conv5 = BatchNormalization()(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)
    conv6 = Conv2D(256, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=3)
    conv7 = Conv2D(128, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)
    conv8 = Conv2D(64, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=3)
    conv9 = Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv9)
    crop9 = Cropping2D(cropping=((16, 16), (16, 16)))(conv9)
    conv9 = BatchNormalization()(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    adam = Adam(lr=1e-3)
    model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])
    return model

In [16]:
# Training sample resolution: 256x256
img_size = 256
batch_size = 64
train_steps = 10000 // batch_size
val_steps = 1000 // batch_size

In [4]:
model = get_unet(3, img_size, img_size)

/anaconda/envs/stanford/lib/python3.6/site-packages/ipykernel_launcher.py:35: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/anaconda/envs/stanford/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/anaconda/envs/stanford/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/anaconda/envs/stanford/lib/python3.6/site-packages/ipykernel_launcher.py:47: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/anaconda/envs/stanford/lib/pyth

# Augmentation

In [5]:
def flip_axis(x, axis):
    x = np.asarray(x).swapaxes(axis, 0)
    x = x[::-1, ...]
    x = x.swapaxes(0, axis)
    return x

In [9]:
# 
path_train = '/Users/Orchestrator/Documents/Ambition/Projects/AngelHackLondon/data/buildings_set/images/train/'
path_valid = '/Users/Orchestrator/Documents/Ambition/Projects/AngelHackLondon/data/buildings_set/images/valid/'
train = [os.path.join(path_train, f ) for f in os.listdir(path_train) if f.endswith('.tiff')]
valid = [os.path.join(path_valid, f ) for f in os.listdir(path_valid) if f.endswith('.tiff')]

In [10]:
# Create the generators
train_generator = batch_generator(train, train_steps, batch_size, (img_size, img_size, 3))
validation_generator = batch_generator(valid, val_steps, batch_size, (img_size, img_size, 3))

In [12]:
reduce_lr = ReduceLROnPlateau(monitor='val_jaccard_coef_int', factor=0.5, patience=1, min_lr=1e-6)
checkpoint = ModelCheckpoint('augmented_weights_{epoch:02d}.hdf5')
callbacks_list = [reduce_lr, checkpoint]

# Train

In [17]:
# To implement model on gpu*
model_fit = model.fit_generator(generator=train_generator, steps_per_epoch=train_steps,
        epochs=1, verbose=1, validation_data=validation_generator, validation_steps=val_steps,
        callbacks=callbacks_list)

Epoch 1/1
156/156 [==============================] - 4775s 31s/step - loss: 3.5692 - binary_crossentropy: 1.0077 - jaccard_coef_int: 0.3012 - val_loss: 3.7580 - val_binary_crossentropy: 1.7605 - val_jaccard_coef_int: 0.2754
